# Training VGG16 from Scratch for Image Classification

This notebook demonstrates how to implement and train a VGG16 model from scratch using PyTorch for binary image classification (Dogs vs Cats). We'll break down the implementation into several key sections,
1. Setting up the environment and importing dependencies
2. Data preparation and loading
3. Model architecture implementation
4. Training utilities and visualization functions
5. Training loop and model evaluation

## 1. Setup and Dependencies

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
import matplotlib.pyplot as plt
import seaborn as sns
import time

In [ ]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Constants
BATCH_SIZE = 16
IMG_SIZE = 224
NUM_EPOCHS = 10

## 2. Data Preparation 

### 2.1 Data Augmentation and Transforms
We'll set up data augmentation for training and basic transforms for validation. Data augmentation helps prevent overfitting and improves model generalization.

In [ ]:
# Training transforms with augmentation
train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.RandomAffine(0, translate=(0.14, 0.14)),
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
# Validation transforms (no augmentation)
val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

### 2.2 Dataset Loading
Now we'll load our datasets using PyTorch's ImageFolder.

In [ ]:
# Load datasets
train_dataset = datasets.ImageFolder(
    root='datasets/train',
    transform=train_transform
)

val_dataset = datasets.ImageFolder(
    root='datasets/val',
    transform=val_transform
)

In [ ]:
print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")

In [ ]:
# Create data loaders
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

## 3. Model Architecture

We'll implement VGG16 from scratch following the original architecture but with modern improvements like batch normalization and dropout.

In [ ]:
class VGG16FromScratch(nn.Module):
    def __init__(self, num_classes=2):
        super(VGG16FromScratch, self).__init__()
        
        # Block 1
        self.block1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        # Block 2
        self.block2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        # Block 3
        self.block3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        # Block 4
        self.block4 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        # Block 5
        self.block5 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        # Classifier
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.BatchNorm1d(512 * 7 * 7),
            nn.Linear(512 * 7 * 7, 256),
            nn.Softplus(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.5),
            
            nn.Linear(256, 256),
            nn.Softplus(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.5),
            
            nn.Linear(256, 256),
            nn.Softplus(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.5),
            
            nn.Linear(256, 256),
            nn.Softplus(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.5),
            
            nn.Linear(256, num_classes),
            nn.Softmax(dim=1)
        )
        
        # Initialize weights
        self._initialize_weights()
    
    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        x = self.classifier(x)
        return x
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

In [ ]:
# Create model instance"
model = VGG16FromScratch().to(device)
# Print model summary,
total_params = sum(p.numel() for p in model.parameters())
print(f'Total parameters: {total_params:,}')

## 4. Training Utilities

### 4.1 Visualization Function

In [ ]:
def plot_training_history(train_losses, val_losses, train_accs, val_accs, title=''):
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(16, 6))
    
    # Loss plot
    ax[0].plot(train_losses, label='Training Loss')
    ax[0].plot(val_losses, label='Validation Loss')
    ax[0].legend(loc='upper right')
    ax[0].set_title(title + ' Loss')
    
    # Accuracy plot
    ax[1].plot(train_accs, label='Training Accuracy')
    ax[1].plot(val_accs, label='Validation Accuracy')
    ax[1].legend(loc='lower right')
    ax[1].set_title(title + ' Accuracy')
    
    plt.show()

### 4.2 Training Function

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs):
    since = time.time()
    best_val_loss = float('inf')
    train_losses, val_losses = [], []
    train_accs, val_accs = [], []
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
        
        train_loss = running_loss / len(train_loader)
        train_acc = 100. * correct / total
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()
        
        val_loss = val_loss / len(val_loader)
        val_acc = 100. * correct / total
        
        # Learning rate scheduling
        scheduler.step(val_loss)
        
         # Update best accuracy
        if val_acc > best_acc:
            best_acc = val_acc
            
        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), 'vgg16_scratch_best.pth')
        
        # Store metrics
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accs.append(train_acc)
        val_accs.append(val_acc)
        
        print(f'Epoch {epoch+1}/{num_epochs}:')
        print(f'Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%')
        print(f'Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%')
        print('-' * 50)

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:.4f}'.format(best_acc))
        
    return train_losses, val_losses, train_accs, val_accs

## 5. Training and Evaluation

In [ ]:
# Define loss function (criterion)
criterion = nn.CrossEntropyLoss()

# Define optimizer with initial learning rate
initial_lr = 0.001
optimizer = optim.Adam(model.parameters(), lr=initial_lr, weight_decay=1e-4)  # Added weight decay for regularization

# Define learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='min',           # Reduce LR when validation loss stops decreasing
    factor=0.1,          # Multiply LR by this factor when plateauing
    patience=3,          # Number of epochs to wait before reducing LR
    verbose=True,        # Print message when LR is reduced
    min_lr=1e-6         # Minimum LR value
)

In [ ]:
# Train the model
train_losses, val_losses, train_accs, val_accs = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    num_epochs=NUM_EPOCHS
)

In [ ]:
# Plot the training results
plot_training_history(train_losses, val_losses, train_accs, val_accs, title='VGG16 Training')

# Load the best model 
model.load_state_dict(torch.load('vgg16_scratch_best.pth'))